In [1]:
cd /home/makinen/repositories/fishnets/

/home/makinen/repositories/fishnets


In [2]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
#import torch
import jax
from jax import lax
#import jax_cosmo as jc
import scipy.constants as cnst
import scipy.stats as ss

import matplotlib.pyplot as plt

import numpy as np
import numpyro
import numpyro.distributions as dist

import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import trange
from scipy import stats
tfk = tf.keras

from fishnets import *

/softs/python/py3mods/tensorflow/2.8.4/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# first get targets for UNCENSORED data
### do seed matching

In [3]:
def gamma_pop_model_censored(key, theta, 
                    n_data=500, 
                    serum_min=25., 
                    tmax=10.,
                    A=500.0):

  population = jnp.zeros((n_data, 2))
  mean, scale = theta # shape, reporting delay
  rate = 1./ scale #concentration / report_delay
  concentration = mean / scale
  key,rng = jax.random.split(key)

  def fn(data_tuple):
    data,key = data_tuple
    key,rng = jax.random.split(key)
    decay_time = dist.Gamma(concentration=concentration, 
                           rate=rate).sample(key, ())

    key,rng = jax.random.split(key)
    measurement_time = dist.Uniform(low=0.0, high=tmax).sample(key, ())

    key,rng = jax.random.split(key)
    _lambda = A * jnp.exp(-measurement_time / decay_time )
    measured_serum_level = dist.Poisson(rate=_lambda).sample(key, ())

    #measured_serum_level += dist.Normal(loc=0.0, scale=0.2).sample(key, ())

    return jnp.array([measurement_time, measured_serum_level]), key

  def cond_fun(data_tuple):
    data,_ = data_tuple
    return data[1] < serum_min

  for i in range(population.shape[0]):
      key,rng = jax.random.split(key)
      #ody_fn = lambda d: fn(key=key, data=d)
      data,_ = jax.lax.while_loop(cond_fun, fn, init_val=(jnp.ones((2,))*0, key))

      population = population.at[i, :].set(data)

  return population

In [4]:
# mean: (0.5, 10)
# scale: (0.1, 1.5)
# theta+fid = [5.0, 0.8]

theta_targets = jnp.array([[5.1, 0.82],
                            [3.0, 0.5],
                            [2.0, 0.8],
                            [8.0, 1.0],

])

target_key = jax.random.PRNGKey(99)
target_keys = jax.random.split(target_key, num=4)

_simulator = lambda k,t: gamma_pop_model_censored(k,t, n_data=500)
target_sims_500 = jax.vmap(_simulator)(target_keys, theta_targets)

_simulator = lambda k,t: gamma_pop_model_censored(k,t, n_data=10000)
target_sims_10k = jax.vmap(_simulator)(target_keys, theta_targets)


# save everything
target_dir = "/data80/makinen/fishnets/gamma_pop/censored_targets/"

np.save(target_dir + "target_keys", target_keys)
np.save(target_dir + "target_sims_500", target_sims_500)
np.save(target_dir + "target_sims_10k", target_sims_10k)


target_sims_500.shape, target_sims_10k.shape


((4, 500, 2), (4, 10000, 2))

In [5]:
# set up fishnets params
n_theta = 2

tmax=10. # days
serum_max_val=4.0

theta_fid = tf.constant([5.0, 0.8], dtype=tf.float32) 
theta_fid_ = theta_fid.numpy()


def get_estimate(model_path, data, n_data, n_sims=4):

    data = np.array(data)
    # preprocess the data
    # make data neural-net friendly
    datamax = 500.
    tmax = 10.

    data[:, :, 0] /= tmax
    data[:, :, 1] /= datamax

    # stack up the data and parameters
    data = tf.convert_to_tensor(data, dtype=tf.float32)

    # construct masks
    score_mask = np.ones((n_sims, n_data, n_theta))
    fisher_mask = np.ones((n_sims, n_data, n_theta, n_theta))


    score_mask = tf.convert_to_tensor(score_mask, dtype=tf.float32)
    fisher_mask = tf.convert_to_tensor(fisher_mask, dtype=tf.float32)

    Model = FishnetTwin(n_parameters=n_theta, 
                n_inputs=2, 
                n_hidden_score=[256, 256, 256], 
                activation_score=[tf.nn.elu, tf.nn.elu,  tf.nn.elu],
                n_hidden_fisher=[256, 256, 256], 
                activation_fisher=[tf.nn.elu, tf.nn.elu,  tf.nn.elu],
                optimizer=tf.keras.optimizers.Adam(lr=5e-4),
                theta_fid=theta_fid,
                priormu=tf.zeros(n_theta, dtype=tf.float32),
                priorCinv=tf.eye(n_theta, dtype=tf.float32),
                restore=True,
                restore_filename=model_path)

    mle, F  = Model.compute_mle_(data, score_mask, fisher_mask)

    return mle, F

In [6]:
# load in Fishnets models to get neural summaries
# load in whole ensemble
parentdir = "/data80/makinen/fishnets/gamma_pop/results/"

mles_500 = []
Fs_500 = []

mles_10k = []
Fs_10k = []

for i in range(10):
    #  load model
    modelpath = parentdir + "model_censored_%d/checkpoint_4/model"%(i)

    # do n_data=500 first
    mle,F = get_estimate(modelpath, target_sims_500, n_data=500, n_sims=4)
    mles_500.append(mle.numpy())
    Fs_500.append(F.numpy())

    print("model %d MSEs, n_data=500: "%(i), (mle.numpy() - theta_targets)**2)

    # now do n_data = 10k
    mle,F = get_estimate(modelpath, target_sims_10k, n_data=10000, n_sims=4)
    mles_10k.append(mle.numpy())
    Fs_10k.append(F.numpy())

    print("model %d MSEs, n_data=10k: "%(i), (mle.numpy() - theta_targets)**2)


np.save(target_dir + "mles_500", np.array(mles_500))
np.save(target_dir + "Fs_500", np.array(Fs_500))

np.save(target_dir + "mles_10k", np.array(mles_10k))
np.save(target_dir + "Fs_10k", np.array(Fs_10k))

/softs/python/py3mods/tensorflow/2.8.4/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


loading model
restoring variables
model 0 MSEs, n_data=500:  [[1.7768252e-05 5.4949475e-03]
 [7.7629469e-05 3.2688831e-03]
 [4.5271777e-02 4.6405059e-04]
 [9.0076644e-03 3.8435536e-03]]
loading model
restoring variables
model 0 MSEs, n_data=10k:  [[0.01710534 0.0006353 ]
 [0.00075663 0.00155147]
 [0.00129009 0.00090796]
 [0.0252087  0.01644453]]
loading model
restoring variables
model 1 MSEs, n_data=500:  [[1.3822463e-04 6.6329641e-03]
 [5.8981466e-05 3.6331927e-03]
 [4.7880981e-02 6.3223776e-04]
 [9.0368325e-03 4.7237878e-03]]
loading model
restoring variables
model 1 MSEs, n_data=10k:  [[0.01712567 0.00063491]
 [0.00062984 0.00167931]
 [0.0014583  0.00066377]
 [0.02406958 0.01658566]]
loading model
restoring variables


model 2 MSEs, n_data=500:  [[2.4105102e-06 5.5006398e-03]
 [4.9481569e-06 3.1228687e-03]
 [4.3644678e-02 1.6017366e-04]
 [1.0508002e-02 4.9133096e-03]]
loading model
restoring variables


model 2 MSEs, n_data=10k:  [[0.01673567 0.00057807]
 [0.00100481 0.00129445]
 [0.00118469 0.00054061]
 [0.02197823 0.01475315]]
loading model
restoring variables
model 3 MSEs, n_data=500:  [[1.2303357e-05 4.6220734e-03]
 [5.1159077e-13 3.8010578e-03]
 [4.3748640e-02 1.4549935e-04]
 [1.0330929e-02 4.4548926e-03]]
loading model
restoring variables
model 3 MSEs, n_data=10k:  [[0.01658819 0.00046695]
 [0.00107577 0.00155707]
 [0.00092926 0.00054696]
 [0.02354398 0.01632722]]
loading model
restoring variables
model 4 MSEs, n_data=500:  [[8.3819032e-07 4.9530887e-03]
 [3.2886579e-05 5.3388923e-03]
 [4.5844782e-02 1.4521190e-04]
 [1.1343196e-02 2.6553965e-03]]
loading model
restoring variables
model 4 MSEs, n_data=10k:  [[0.01675381 0.00052833]
 [0.00121782 0.00106695]
 [0.00119784 0.00073099]
 [0.02423676 0.01506498]]
loading model
restoring variables
model 5 MSEs, n_data=500:  [[1.3406306e-04 6.3615744e-03]
 [5.3100739e-05 3.6000146e-03]
 [4.8434846e-02 4.3785750e-04]
 [1.0313489e-02 2.9673

In [7]:
parentdir

'/data80/makinen/fishnets/gamma_pop/results/'

In [9]:
datadir = "/data80/makinen/fishnets/gamma_pop/data/"

training_theta_10k = np.load(datadir + 'theta_censored_ndata_10k.npy') # this is the correct length
training_theta_500 = np.load(datadir + 'theta_gamma_censored.npy')

training_theta_10k.shape, training_theta_500.shape

((5000, 2), (100000, 2))

In [10]:
target_dir

'/data80/makinen/fishnets/gamma_pop/censored_targets/'

In [16]:
# load and package all delfi training data for each model
training_mle_500 = []
#training_theta_500 = np.load(parentdir + 'model_censored_0/theta.npy')

training_mle_10k = []
#training_theta_10k = np.load(parentdir + 'model_censored_0/test_theta.npy')


for i in range(10):
    modelpath = parentdir + "model_censored_%d/checkpoint_4/"%(i)

    mle = np.load(modelpath + '/mle.npy', )
    training_mle_500.append(mle)
    
    mle = np.load(parentdir + 'model_censored_%d/'%(i) + 'test_mle.npy' )
    training_mle_10k.append(mle)

np.save(target_dir + 'training_mle_500', np.array(training_mle_500))
np.save(target_dir + 'training_theta_500', training_theta_500)


np.array(training_mle_500).shape, training_theta_500.shape, np.array(training_mle_10k).shape, training_theta_10k.shape, 

((10, 100000, 2), (100000, 2), (10, 1000, 2), (5000, 2))

# generate some more sims of 10k

In [19]:
# n_sims = 5000
# # theta = (concentration, rate)
# # mean of Gamma = concentration / rate
# key = jax.random.PRNGKey(33)
# mean = dist.Uniform(low=0.5, high=10.).sample(key, (n_sims,))
# key,rng = jax.random.split(key)
# scale = dist.Uniform(low=0.1, high=1.5).sample(key, (n_sims,))


# theta = jnp.vstack([mean, scale]).T

# keys = jax.random.split(jax.random.PRNGKey(0), num=n_sims)
# _simulator = lambda k,t: gamma_pop_model(k,t, n_data=10000)
# training_sims_10k = jax.vmap(_simulator)(keys, theta)

# # save everything
# target_dir = "/data80/makinen/fishnets/gamma_pop/uncensored_targets/"

# np.save(target_dir + "training_sims_10k", training_sims_10k)
# np.save(target_dir + "training_theta_10k", theta)

# training_sims_10k.shape

(5000, 10000, 2)

In [20]:
# now for 10k
datadir = '/data80/makinen/fishnets/gamma_pop/data/'

training_sims_10k = np.load(datadir + 'data_censored_ndata_10k.npy')
training_theta_10k = np.load(datadir + 'theta_censored_ndata_10k.npy')

training_sims_10k.shape, training_theta_10k.shape

((5000, 10000, 2), (5000, 2))

In [21]:
# load in Fishnets models to get neural summaries
# load in whole ensemble
parentdir = "/data80/makinen/fishnets/gamma_pop/results/"

mles_training_10k = []

for i in range(10):
    #  load model
    modelpath = parentdir + "model_censored_%d/checkpoint_4/model"%(i)

    # do n_data=500 first
    mle,_ = get_estimate(modelpath, training_sims_10k, n_data=10000, n_sims=5000)
    mles_training_10k.append(mle.numpy())



np.save(target_dir + "training_mles_10k", np.array(mles_training_10k))
np.save(target_dir + "training_theta_10k", training_theta_10k)

In [21]:
np.array(mles_training_10k).shape

(10, 5000, 2)

# get estimates for new loss function

In [31]:
# set up fishnets params
n_theta = 2

tmax=10. # days
serum_max_val=4.0

theta_fid = tf.constant([5.0, 0.8], dtype=tf.float32) 
theta_fid_ = theta_fid.numpy()


def get_estimate(model_path, data, n_data, n_sims=4):

    data = np.array(data)
    # preprocess the data
    # make data neural-net friendly
    datamax = 500.
    tmax = 10.

    data[:, :, 0] /= tmax
    data[:, :, 1] /= datamax

    # stack up the data and parameters
    data = tf.convert_to_tensor(data, dtype=tf.float32)



    # construct masks
    score_mask = np.ones((n_sims, n_data, n_theta))
    fisher_mask = np.ones((n_sims, n_data, n_theta, n_theta))


    score_mask = tf.convert_to_tensor(score_mask, dtype=tf.float32)
    fisher_mask = tf.convert_to_tensor(fisher_mask, dtype=tf.float32)

    Model = FishnetTwin(n_parameters=n_theta, 
                n_inputs=2, 
                n_hidden_score=[256, 256, 256], 
                activation_score=[tf.nn.elu, tf.nn.elu,  tf.nn.elu],
                n_hidden_fisher=[256, 256, 256], 
                activation_fisher=[tf.nn.elu, tf.nn.elu,  tf.nn.elu],
                optimizer=tf.keras.optimizers.Adam(lr=5e-4),
                theta_fid=theta_fid,
                priormu=tf.zeros(n_theta, dtype=tf.float32),
                priorCinv=tf.eye(n_theta, dtype=tf.float32),
                restore=True,
                restore_filename=model_path)

    # add in corrected loss function
    @tf.function
    def construct_fisher_matrix(outputs):
        
        Q = tfp.math.fill_triangular(outputs)
        # EDIT: changed to + softplus(diag_part(Q))
        L = Q - tf.linalg.diag(tf.linalg.diag_part(Q) - tf.math.softplus(tf.linalg.diag_part(Q)))
        return tf.einsum('...ij,...jk->...ik', L, tf.transpose(L, perm=[0, 1, 3, 2]))

    Model.construct_fisher_matrix = construct_fisher_matrix

    mle, F  = Model.compute_mle_(data, score_mask, fisher_mask)

    return mle, F

In [34]:
# load everything
target_dir = "/data80/makinen/fishnets/gamma_pop/uncensored_targets/"

#np.load(target_dir + "target_keys", target_keys)
target_sims_500 = np.load(target_dir + "target_sims_500.npy")
target_sims_10k = np.load(target_dir + "target_sims_10k.npy")

In [37]:
# load in Fishnets models to get neural summaries
# load in whole ensemble
parentdir = "/data80/makinen/fishnets/gamma_pop/results/"

mles_500 = []
Fs_500 = []

mles_10k = []
Fs_10k = []

for i in range(3):
    #  load model
    modelpath = parentdir + "model_new_%d/checkpoint_4/model"%(i)

    # do n_data=500 first
    mle,F = get_estimate(modelpath, target_sims_500, n_data=500, n_sims=4)
    mles_500.append(mle.numpy())
    Fs_500.append(F.numpy())

    #print("model %d MSEs, n_data=500: "%(i), (mle.numpy() - theta_targets)**2)

    # now do n_data = 10k
    mle,F = get_estimate(modelpath, target_sims_10k, n_data=10000, n_sims=4)
    mles_10k.append(mle.numpy())
    Fs_10k.append(F.numpy())

    #print("model %d MSEs, n_data=10k: "%(i), (mle.numpy() - theta_targets)**2)


np.save(target_dir + "mles_new_500", np.array(mles_500))
np.save(target_dir + "Fs_new_500", np.array(Fs_500))

np.save(target_dir + "mles_new_10k", np.array(mles_10k))
np.save(target_dir + "Fs_new_10k", np.array(Fs_10k))

loading model
restoring variables
loading model
restoring variables
loading model
restoring variables
loading model
restoring variables
loading model
restoring variables
loading model
restoring variables


In [41]:
np.load(target_dir + 'Fs_10k.npy')

array([[[[ 2.1731221e+03, -1.6154799e+01],
         [-1.6154799e+01,  9.8573743e+02]],

        [[ 2.3752612e+03, -4.6494104e+02],
         [-4.6494104e+02,  7.6216443e+02]],

        [[ 2.6383420e+03, -7.2758179e+02],
         [-7.2758179e+02,  6.4276801e+02]],

        [[ 2.2096716e+03,  4.4178644e+02],
         [ 4.4178644e+02,  1.1315194e+03]]],


       [[[ 2.1663982e+03, -2.0142527e+01],
         [-2.0142527e+01,  9.7478949e+02]],

        [[ 2.3662129e+03, -4.6657794e+02],
         [-4.6657794e+02,  7.5508069e+02]],

        [[ 2.6326013e+03, -7.3103394e+02],
         [-7.3103394e+02,  6.4046997e+02]],

        [[ 2.2018975e+03,  4.2959955e+02],
         [ 4.2959955e+02,  1.1157897e+03]]],


       [[[ 2.2041943e+03,  6.3981237e+00],
         [ 6.3981237e+00,  9.7821198e+02]],

        [[ 2.3727341e+03, -4.1451956e+02],
         [-4.1451956e+02,  7.0508496e+02]],

        [[ 2.5751685e+03, -6.4667578e+02],
         [-6.4667578e+02,  5.7827820e+02]],

        [[ 2.2373767e+03,  4

In [42]:
np.array(Fs_500)

array([[[[106.15389   ,   0.95012665],
         [  0.95012665,  48.4604    ]],

        [[119.78084   , -22.853203  ],
         [-22.853203  ,  38.789127  ]],

        [[129.72758   , -33.81417   ],
         [-33.81417   ,  32.78931   ]],

        [[110.82899   ,  19.824207  ],
         [ 19.824207  ,  56.470917  ]]],


       [[[107.40982   ,   0.48831248],
         [  0.48831248,  48.3251    ]],

        [[121.982475  , -23.545845  ],
         [-23.545845  ,  39.71639   ]],

        [[132.13809   , -34.94921   ],
         [-34.94921   ,  33.873245  ]],

        [[112.11726   ,  19.261057  ],
         [ 19.261057  ,  56.2925    ]]],


       [[[108.429436  ,   1.0730045 ],
         [  1.0730045 ,  49.99024   ]],

        [[122.88202   , -23.462118  ],
         [-23.462118  ,  39.56526   ]],

        [[132.85994   , -34.31184   ],
         [-34.31184   ,  33.10341   ]],

        [[113.34428   ,  20.778212  ],
         [ 20.778212  ,  58.70424   ]]]], dtype=float32)

In [32]:
# load 10k training sims
training_sims_10k = np.load('/data80/makinen/fishnets/gamma_pop/uncensored_targets/training_sims_10k.npy')
theta_test = np.load('/data80/makinen/fishnets/gamma_pop/uncensored_targets/training_theta_10k.npy')

In [43]:
# load in Fishnets models to get neural summaries
# load in whole ensemble
parentdir = "/data80/makinen/fishnets/gamma_pop/results/"

mles_training_10k = []

for i in range(3):
    #  load model
    modelpath = parentdir + "model_new_%d/checkpoint_4/model"%(i)

    # do n_data=500 first
    mle,_ = get_estimate(modelpath, training_sims_10k, n_data=10000, n_sims=5000)
    mles_training_10k.append(mle.numpy())



np.save(target_dir + "mles_new_training_10k", np.array(mles_training_10k))